In [66]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.linear_model import LogisticRegression, LinearRegression

pd.set_option("display.max_columns", 100)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import itertools
sns.set()

plt.style.use('ggplot')

In [67]:
df = pd.read_csv('nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv')
df = df[np.isfinite(df['down'])]


/Users/reiffd/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (42,167,168,169,170,179,180,183,184,189,190,191,192,199,200,201,202,203,204,205,206,208,210,212,227,228,229,230,231,232,233,234,235,237,238,239,241,242,250) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
# def yards_per_drive(series):
#     return reduce(lambda x, y: x if (x > y) else y, series)

In [41]:
def yards_per_drive(series):
    return max(series) - min(series)

# def first_and_tens(group):
#     count = 0
#     down = group['down']
#     togo = group['ydstogo']
#     if (down == 1) and (togo == 10):
#         count +=1

In [71]:
# df.groupby(['game_id', 'drive', 'posteam', 'defteam']).agg({'yardline_100': [yards_per_drive], 'down': 'max'})
# df.groupby(['game_id', 'drive', 'posteam', 'defteam']).agg({'down': lambda x: x['down'][(x['down']==1) & (x['ydstogo']==10)].count()})

# df.groupby(['game_id', 'drive', 'posteam', 'defteam'])['down'].aggregate(lambda x: x[(x['down']>1) & (x['ydstogo']==10)].count())

df.groupby(['game_id', 'drive', 'posteam', 'defteam']) \
  .apply(lambda x: pd.Series({
      'drive_yards' : x['yardline_100'].max() - x['yardline_100'].min(),
      'longest_play': x['yards_gained'].max(),
      'num_shotgun' : x['shotgun'].count(),
      'num_third_down': x['down'][x['down'] == 3].count(),
      'pass_on_first' : x['down'][(x['down'] == 1) & (x['play_type'] == 'pass')].count(),
      'pass_on_second': x['down'][(x['down'] == 2) & (x['play_type'] == 'pass')].count(),
      'third_down_conversions': x['down'][(x['down'] == 3) & (x['yards_gained'] >= x['ydstogo'])].count()
  })
)


drive_yards  longest_play  num_shotgun  \
game_id    drive posteam defteam                                           
2018090600 1     ATL     PHI             79.0          33.0         11.0   
           2     PHI     ATL              6.0           4.0          4.0   
           3     ATL     PHI             41.0          26.0         10.0   
           4     PHI     ATL             12.0           6.0          8.0   
           5     ATL     PHI              9.0           0.0          4.0   
...                                       ...           ...          ...   
2018123015 23    SEA     ARI             20.0           7.0          6.0   
           24    ARI     SEA             22.0          15.0          6.0   
           25    SEA     ARI             13.0          12.0          6.0   
           26    ARI     SEA             41.0          18.0          9.0   
           27    SEA     ARI             63.0          37.0          8.0   

                                  num_third_down  pass_on_first  \
game_id    drive posteam defteam                                  
2018090600 1     ATL     PHI                 2.0            2.0   
           2     PHI     ATL                 1.0            0.0   
           3     ATL     PHI                 3.0            2.0   
           4     PHI     ATL                 2.0            1.0   
           5     ATL     PHI                 1.0            1.0   
...                                          ...            ...   
2018123015 23    SEA     ARI                 1.0            0.0   
           24    ARI     SEA                 1.0            0.0   
           25    SEA     ARI                 1.0            1.0   
           26    ARI     SEA                 2.0            2.0   
           27    SEA     ARI                 1.0            1.0   

                                  pass_on_second  third_down_conversions  
game_id    drive posteam defteam                                          
2018090600 1     ATL     PHI                 1.0                     1.0  
           2     PHI     ATL                 1.0                     0.0  
           3     ATL     PHI                 2.0                     1.0  
           4     PHI     ATL                 1.0                     0.0  
           5     ATL     PHI                 0.0                     0.0  
...                                          ...                     ...  
2018123015 23    SEA     ARI                 1.0                     0.0  
           24    ARI     SEA                 1.0                     0.0  
           25    SEA     ARI                 1.0                     0.0  
           26    ARI     SEA                 1.0                     1.0  
           27    SEA     ARI                 1.0                     1.0  

[5728 rows x 7 columns]

In [60]:
df = pd.DataFrame(np.random.rand(4,4), columns=list('abcd'))
df['group'] = [0, 0, 1, 1]
df

,a,b,c,d,group
0,0.618114,0.494710,0.107070,0.074995,0
1,0.235047,0.306182,0.591323,0.972107,0
2,0.797402,0.006359,0.468947,0.006625,1
3,0.410568,0.109090,0.473530,0.080861,1


In [61]:
df.groupby('group') \
  .apply(lambda x: pd.Series({
      'a_sum'       : x['a'].sum(),
      'a_max'       : x['a'].max(),
      'b_mean'      : x['b'].mean(),
      'c_d_prodsum' : (x['c'] * x['d']).sum()
  })
)

,a_sum,a_max,b_mean,c_d_prodsum
group,,,,
0,0.853161,0.618114,0.400446,0.582858
1,1.207970,0.797402,0.057725,0.041397
